# ML Classification of HC vs. MCI

This notebook implements a machine learning pipeline for model optimization and evaluation, with the objective to classify participants between HC (healthy controls) and MCI (Mild Cognitive Impairment). Classification is based on GDS: MCI for GDS ≥ 3, healthy otherwise.

Feature selection and ROC curve analysis is performed.




## Summary

1.  **Model Selection**: The pipeline evaluates four different models. Feel free to add/remove any models in the `estimators` list:

    -   Stochastic Gradient Descent Classifier (Logistic Regression)
    -   Support Vector Machine (Linear kernel)
    -   Multi-layer Perceptron
    -   Voting Classifier (ensemble of the above three)

2.  **Cross-Validation**: Uses Leave-One-Group-Out cross-validation,
    which is appropriate when you have multiple observations per
    subject/group.

3.  **Feature Selection**: Employs permutation importance for feature
    selection, which is model-agnostic and works by measuring the
    decrease in model performance when a feature is randomly shuffled.

4.  **Evaluation Metrics**: Uses Area Under the ROC Curve (AUC) as the
    primary metric for model evaluation.

5.  **Visualization**: Creates two types of plots:

    -   Feature importance and AUC vs. # of features, for each model
    -   ROC curves comparing all models

6.  **Output**: Saves results in the `results/` directory:

    -   Best features for each model (CSV)
    -   Feature importance plots (PNG)
    -   Combined ROC curves (PNG)


## Setup and Imports

In [ ]:
# Standard libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sklearn classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, CategoricalNB, BernoulliNB
from sklearn.ensemble import (VotingClassifier, RandomForestClassifier, AdaBoostClassifier,
                            GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier)
from sklearn.neural_network import MLPClassifier

# Sklearn utilities
from sklearn.inspection import permutation_importance
from sklearn.model_selection import LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import (StandardScaler, RobustScaler, MinMaxScaler,
                                 PolynomialFeatures, MaxAbsScaler, Normalizer)

## Helper Functions

Define a function to visualize feature importances and their relationship with AUC scores:

In [ ]:
def plot_feature_importances(fis, auc_list, name):
    """
    Create a dual-axis plot showing feature importances and AUC scores.

    Parameters:
    -----------
    fis : numpy.ndarray
        Array of average feature importance scores
    auc_list : list
        List of AUC scores for different numbers of features
    name : str
        Model name for the output file

    Output:
    -------
    Saves a plot showing feature importances (bars) and AUC scores (line)
    with annotation for the optimal number of features
    """
    # Create figure with two y-axes
    fig, ax1 = plt.subplots()

    # First y-axis: Feature Importances (bars)
    color = 'tab:blue'
    ax1.set_xlabel('Features')
    ax1.set_ylabel('AVG Importances', color=color)
    ax1.bar(range(1, len(auc_list) + 1), fis, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    # Second y-axis: AUC scores (line)
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('AUC', color=color)
    ax2.set_ylim(0, 1)
    ax2.plot(range(1, len(auc_list) + 1), auc_list, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    # Add annotation for optimal point
    xmax = auc_list.index(max(auc_list)) + 1
    ymax = max(auc_list)
    text = f"# Features = {xmax}, AUC = {ymax:.3f}"

    # Annotation styling
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops = dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data', textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top")

    # Add annotation
    ax2.annotate(text, xy=(xmax, ymax), xytext=(0.94, 0.96), **kw)

    # Save plot
    fig.tight_layout()
    plt.savefig(f"results/feature_importances_{name}.png")
    plt.clf()

## Model Definition

Define the machine learning models to evaluate. For file management purposes, we give each of them a short name:

In [ ]:
# Define classifier configurations
estimators = [
    {
        'model': SGDClassifier(loss='log_loss'),
        'name': 'logit',
    },
    {
        'model': SVC(kernel='linear', probability=True),
        'name': 'svm',
    },
    {
        'model': MLPClassifier(max_iter=100000),
        'name': 'mlp',
    },
    {
        'model': VotingClassifier(
            estimators=[
                ('sgdc', SGDClassifier(loss='log_loss')),
                ('lsvc', SVC(kernel='linear', probability=True)),
                ('mlpc', MLPClassifier(max_iter=100000))
            ],
            voting='soft'
        ),
        'name': 'vote',
    }
]

## Data Loading and Preprocessing

Two datasets are loaded: `digimoca_results_totales.csv` and `datos_participantes_totales.csv`. These datasets are merged to create a unified dataset for analysis.

We obtain the feature matrix (X) and target variable (y) for MCI detection. Classification is based on GDS: MCI for GDS ≥ 3, healthy otherwise.


In [ ]:
# Load datasets
dataset = pd.read_csv("digimoca_results_totales.csv")
participants = pd.read_csv("datos_participantes_totales.csv")

# Merge datasets
merged = pd.merge(participants, dataset, on='id')

# Extract IDs for cross-validation grouping
ids = merged['id']

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Prepare feature matrix
X = merged.drop(['GDS', 'id', 'gender', 'age', 'studies',
                'lawton', 'MFE', 'T-MoCA', 'timestamp'], axis=1)

# Scale features
scaler = RobustScaler()
X = pd.DataFrame(
    scaler.fit_transform(X),
    columns=scaler.get_feature_names_out()
)

# Prepare target variable (binary classification: GDS >= 3)
y = merged['GDS'].transform(lambda gds: (0 if gds < 3 else 1)).values

## Feature Selection and Model Evaluation

Perform feature selection and evaluation for each model. We sort all the features based on their *permutation importance* and then we evaluate all subsets of features including the N most important (varying the N from 1 to all):


In [ ]:
# Iterate through models
for model in estimators:
    # Fit model
    model['model'].fit(X, y)

    # Calculate feature importance using permutation importance
    result = permutation_importance(
        model['model'], X, y,
        scoring='roc_auc',
        n_repeats=100,
        n_jobs=-1
    )

    # Convert to pandas Series for easier handling
    avg_fi = pd.Series(result.importances_mean, index=X.columns.tolist())

    # Evaluate different numbers of features
    auc_list = []
    for i in range(1, X.shape[1] + 1):
        # Select top i features
        X_best = X[avg_fi.nlargest(i).index]

        # Get cross-validated predictions
        y_pred = cross_val_predict(
            model['model'], X_best, y,
            cv=logo.split(merged, groups=ids),
            n_jobs=-1
        )

        # Calculate and store AUC score
        auc_list.append(roc_auc_score(y, y_pred))

    # Print optimal number of features and corresponding AUC
    print(f"{model['name']}: {auc_list.index(max(auc_list)) + 1} features, "
          f"AUC = {max(auc_list):.3f}")

    # Save best features
    avg_fi.nlargest(auc_list.index(max(auc_list)) + 1).to_csv(
        f"results/best_features_{model['name']}.csv"
    )

    # Plot feature importances
    plot_feature_importances(
        avg_fi.sort_values(ascending=False).values,
        auc_list,
        model['name']
    )

## ROC Curve Generation

Finally, generate ROC curves for all models using only the optimal set of features:

In [ ]:
# Create plot
fig, ax = plt.subplots()

# Generate ROC curves for each model
for model in estimators:
    # Load best features for this model
    best_features = pd.read_csv(
        f"results/best_features_{model['name']}.csv",
        index_col=0
    )

    # Select best features
    X_best = X[best_features.index]

    # Get probability predictions
    y_pred = cross_val_predict(
        model['model'], X_best, y,
        cv=logo.split(merged, groups=ids),
        n_jobs=-1,
        method='predict_proba'
    )

    # Plot ROC curve
    RocCurveDisplay.from_predictions(
        y,
        y_pred[:, 1],
        name=model['name'],
        ax=ax,
        alpha=0.8,
        plot_chance_level=(model['name'] == 'vr')
    )
    ax = plt.gca()

# Set labels
ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate"
)

# Save plot
plt.savefig(f"results/roc_{'_'.join([model['name'] for model in estimators])}.png")